# Docker storage

``````
! sudo tree -L 1 /var/lib/docker

/var/lib/docker
├── buildkit
├── containers
├── engine-id
├── image
├── network
├── overlay2
├── plugins
├── runtimes
├── swarm
├── tmp
└── volumes
``````

Управление данными в Docker
По умолчанию все файлы, созданные внутри контейнера, хранятся на записываемом слое контейнера. Это означает, что:

1. Данные не сохраняются, когда контейнер больше не существует, и может быть сложно извлечь данные из контейнера, если они понадобятся другому процессу.
2. Записываемый слой контейнера тесно связан с хост-машиной, на которой запущен контейнер. Вы не можете легко переместить данные куда-либо еще.
3. Запись в записываемый слой контейнера требует драйвера хранилища для управления файловой системой. 
*Драйвер хранилища предоставляет объединенную файловую систему, используя ядро ​​Linux. Эта дополнительная абстракция снижает производительность по сравнению с использованием томов данных, которые записывают напрямую в файловую систему хоста.*

Docker имеет два варианта для контейнеров для хранения файлов на хост-машине, так что файлы сохраняются даже после остановки контейнера: 
- тома (**volumes**),
- привязки монтирования (**bind mounts**).



Выберите правильный тип монтирования
Независимо от типа монтирования данные выглядят одинаково внутри контейнера. Они отображаются как каталог или отдельный файл в файловой системе контейнера.


### Типы монтирований и где они находятся на хосте Docker
Тома (**volumes**) хранятся в части файловой системы хоста, которая управляется Docker (/var/lib/docker/volumes/ на Linux). Процессы, не относящиеся к Docker, не должны изменять эту часть файловой системы. Тома — лучший способ сохранения данных в Docker.

Монтирования привязки (**bind mounts**) могут храниться в любом месте на хосте. Они могут быть даже важными системными файлами или каталогами. Не-Docker-процессы на хосте Docker или контейнере Docker могут изменять их в любое время.

### Команды для работы с томами:

- **Создание тома**:

  ```
  docker volume create <имя_тома>
  ```

- **Список всех томов**:

  ```
  docker volume ls
  ```

- **Удаление тома**:

  ```
  docker volume rm <имя_тома>
  ```

- **Использование тома при запуске контейнера**:

  ```
  docker run -v <имя_тома>:<путь_в_контейнере> <образ>
  ```


### Команды для монтирования директорий:

- **Монтирование при запуске контейнера**:

  ```
  docker run -v <путь_на_хосте>:<путь_в_контейнере> <образ>
  ```

### Примеры использования:

#### Пример с томом:

```bash
# Создание тома
docker volume create mydata

# Запуск контейнера с использованием тома
docker run -d -v mydata:/app/data nginx
```

В этом примере создается том `mydata`, который используется для хранения данных, доступных для контейнера `nginx` по пути `/app/data`.

#### Пример с монтированием хост-директории:

```bash
# Запуск контейнера с монтированием директории хоста
docker run -d -v /host/path:/container/path nginx
```

В этом примере директория `/host/path` на хосте монтируется внутрь контейнера `nginx` по пути `/container/path`, что позволяет контейнеру работать с данными, хранящимися на хосте.


Пример запуска бд с установкой пароля и монтированием директории:

In [6]:
! docker run -v /opt/data:/var/lib/mysql -d --name mysql-db -e MYSQL_ROOT_PASSWORD=db_pass123 mysql

Unable to find image 'mysql:latest' locally
latest: Pulling from library/mysql

0b27c30f: Pulling fs layer 
01031aab: Pulling fs layer 
c34c4347: Pulling fs layer 
de985fa2: Pulling fs layer 
8a9482de: Pulling fs layer 
786815dd: Pulling fs layer 
ac98ea83: Pulling fs layer 
505c3ae4: Pulling fs layer 
de39aab9: Pulling fs layer 
Digest: sha256:d8df069848906979fd7511db00dc22efeb0a33a990d87c3c6d3fcdafd6fc6123K
Status: Downloaded newer image for mysql:latest
24eb9434122e80233cf7d872c9cd8f4a0a6bcc63f1b56568ed56f8c5cb50417e


In [8]:
! docker ps
! docker inspect mysql-db

CONTAINER ID   IMAGE     COMMAND                  CREATED          STATUS          PORTS                 NAMES
24eb9434122e   mysql     "docker-entrypoint.s…"   32 seconds ago   Up 30 seconds   3306/tcp, 33060/tcp   mysql-db


[
    {
        "Id": "24eb9434122e80233cf7d872c9cd8f4a0a6bcc63f1b56568ed56f8c5cb50417e",
        "Created": "2024-08-08T12:18:38.66123398Z",
        "Path": "docker-entrypoint.sh",
        "Args": [
            "mysqld"
        ],
        "State": {
            "Status": "running",
            "Running": true,
            "Paused": false,
            "Restarting": false,
            "OOMKilled": false,
            "Dead": false,
            "Pid": 12226,
            "ExitCode": 0,
            "Error": "",
            "StartedAt": "2024-08-08T12:18:40.110037061Z",
            "FinishedAt": "0001-01-01T00:00:00Z"
        },
        "Image": "sha256:7ce93a845a8a98c89bd58a5e45fe71277ca5f52177c88046858a6a0af969ba74",
        "ResolvConfPath": "/var/lib/docker/containers/24eb9434122e80233cf7d872c9cd8f4a0a6bcc63f1b56568ed56f8c5cb50417e/resolv.conf",
        "HostnamePath": "/var/lib/docker/containers/24eb9434122e80233cf7d872c9cd8f4a0a6bcc63f1b56568ed56f8c5cb50417e/hostname",
        "HostsPa